In [47]:
import numpy as np

import torch
import torch.nn as nn

In [48]:
class mlp(nn.Module):
    def __init__(self, input_dim=1, hl1_dim=64, hl2_dim = 64, output_dim=1):
        super(mlp, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hl1_dim), 
            nn.Tanh(),
            nn.Linear(hl1_dim, hl2_dim), 
            nn.Tanh(),
            nn.Linear(hl2_dim, output_dim),
            )

    def forward(self, x):
        return self.layers(x)

In [49]:
def task_a(a_min = 0.1, a_max = 5.0):
    a = np.random.rand()
    a = a * (a_max - a_min) + a_min
    
    phase = 2 * np.pi * np.random.rand()
    
    #Using torch linspace and sin so that it makes tensors
    x_tau = torch.linspace(-5, 5, 50)
    y_tau = a * torch.sin(x_tau + phase)

    return x_tau, y_tau

In [57]:
model = mlp()
batchsize = 10
lr = 0.02
loss_calc = nn.MSELoss()


optim = torch.optim.SGD(model.parameters(), lr=lr)
#Can use SGD or Adam optimiser, using SGD currently
#optim = torch.optim.Adam(model.parameters(), lr=lr)


x_tau, y_tau = task_a()

for batch_iter in range(x_tau.shape[0] // batchsize):
    index_train_slice = torch.randperm(x_tau.shape[0])[batch_iter*batchsize:(batch_iter*batchsize)+batchsize]

    batch_in = x_tau[index_train_slice].reshape(-1,1)
    batch_target = y_tau[index_train_slice].reshape(-1,1)

    loss = loss_calc(model(batch_in), batch_target)
    optim.zero_grad()
    loss.backward()
    optim.step()  